# Comparison of different SEI models

See impace of different available SEI options.

In [1]:
import pybamm

* Creating a simple experiment with 100% discharge and 100% charging cylcles. Running these cycles for 30 times.

* Using `Chen2020` parameter values.

In [2]:
my_exp = pybamm.Experiment(
    ["Discharge at 1C for 1 hour",
    "Charge at 1C for 1 hour"] * 30
)

my_param = pybamm.ParameterValues("Chen2020")

sols = []

* Make a list of all SEI options to explore.

In [3]:
sei_options = ["none", "constant", "reaction limited", "solvent-diffusion limited"]

* Iterate through the list and apply each option one by one to the model.

* `model.name` is used to give name to the model to be used while plotting.

In [4]:
for option in sei_options:
    model = pybamm.lithium_ion.DFN(options={"SEI": option})
    model.name = option
    sim = pybamm.Simulation(model, experiment=my_exp, parameter_values=my_param)
    sols.append(sim.solve())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning: The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.
  self._solver = solver or self._model.default_solver


* Use the `model.varaibles.search()` function to search for appropriate varaibles to plot

In [5]:
model.variables.search("inventory")

Results for 'inventory': ['Loss of lithium inventory [%]', 'Loss of lithium inventory, including electrolyte [%]']


In [6]:
pybamm.dynamic_plot(sols, output_variables=['X-averaged negative total SEI thickness [m]', 'X-averaged positive total SEI thickness [m]', "Loss of lithium inventory [%]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=60.0, step=0.6), Output()), _dom_classes=('w…

![Alt text](../images/sei.png)

### Plot Explanation

1. X-avg negative SEI Thickness:
* `none`, there is no SEI present

* `constant`, there is a constant SEI thickness which does not grow through the experiment. SEI thickness is predefined in the parameter values.

* `reaction limited`, it starts with the predefined SEI thickness and increases because RDS is the solvent availability. This option is used to model initial SEI growth so SEI grows untill solvent is present.

* `solvent-diffusion limited`, there is very litte growth of SEI as it is limited by diffusion of solvent across SEI to reach -ve electrode. This is used to model degradation and capacity fade.


2. X-avg positive SEI Thickness:
* no SEI is grown on positive electrode


3. LLI
* Clearly seen that `reaction limited` consumes the most Li-inventory as it models initial SEI growth followed by `solvent-diffusion limited`. Other two options do not consume Li ions.